### Task 1: Classifying "Good" vs. "Bad" Tracks

In [5]:
import os
import sys
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import joblib
import torch.nn.init as init
import seaborn as sns

from torch.utils.data import WeightedRandomSampler
from PIL import Image
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.model_selection import StratifiedKFold

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_dir = "/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/Sorted/path_imgdata"

#### Image pre-processing and data setup

In [7]:
def customcrop(image):
    return transforms.functional.crop(image, 70, 155, 700, 875)

In [8]:
data_transform = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),
#     transforms.Resize((50, 50)),
    transforms.Lambda(customcrop),
    transforms.Resize((100,125)),
    transforms.ToTensor()]
)
dataset = torchvision.datasets.ImageFolder(root_dir, transform=data_transform)

FileNotFoundError: [Errno 2] No such file or directory: '/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/Sorted/path_imgdata'

In [9]:
total_n = len(dataset)
train_n = int(0.8 * total_n)
test_n = total_n - train_n

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_n, test_n)
)

NameError: name 'dataset' is not defined

In [10]:
train_dataset.dataset.class_to_idx

NameError: name 'train_dataset' is not defined

In [11]:
y_trnidx = train_dataset.indices
y_trn = [dataset.targets[i] for i in y_trnidx]
class_n = np.array([np.sum(y_trn == c) for c in np.unique(y_trn)])

NameError: name 'train_dataset' is not defined

In [12]:
### optional sampling of data, not necessary
# weight = 1. / class_n
# samples_weight = np.array([weight[t] for t in y_trn])
# samples_weight = torch.from_numpy(samples_weight)
# smplr = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))
# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=8, num_workers=2, pin_memory=True, sampler=smplr
#     )

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, num_workers=2, shuffle=True, pin_memory=True
    )

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

print(len(train_loader))

NameError: name 'train_dataset' is not defined

#### Example of a flight path/track

In [15]:
plt.imshow((next(iter(train_loader))[0][0]).permute(1, 2, 0))
plt.show()

NameError: name 'train_loader' is not defined

#### Defining the model

In [16]:
model = models.resnet50(pretrained=False)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = model.to(device)

In [17]:
n_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)

In [18]:
def test(loader, criterion):
    model.eval()
    running_loss = 0.
    total = 0
    correct = 0

    for images,labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            pred = model(images)
            loss = criterion(pred, labels)
            
        running_loss += loss
        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        
    val_acc = correct/total
    val_loss = running_loss/len(loader)
    model.train()
    
    return val_acc, val_loss

#### Training the model

In [19]:
trn_losses = []
trn_accs = []
val_losses = []
val_accs = []

for epoch in range(n_epochs):
    loss_avg = 0.
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader)
    for i, (images,labels) in enumerate(progress_bar):
        progress_bar.set_description("Epoch " + str(epoch))
        images = images.to(device)
        labels = labels.to(device)
        
        pred = model(images)
        loss = criterion(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_avg += loss.item()
        
        pred = torch.max(pred.data,1)[1]
        total += labels.size()[0]
        correct += (pred == labels.data).sum().item()
        accuracy = correct/total
        
        progress_bar.set_postfix(acc=accuracy)
    
    trn_losses.append(loss_avg/i)
    trn_accs.append(accuracy)
    
    test_acc, test_loss = test(test_loader, criterion)
    val_accs.append(test_acc)
    val_losses.append(test_loss)
    tqdm.write("test_acc: %.3f" % test_acc)    

NameError: name 'train_loader' is not defined

#### Saving the end model and its weights

In [20]:
torch.save(model.state_dict(), "visual_only_resnet50.pth")